In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

In [7]:
from sqlalchemy import inspect

# Reflect Tables into SQLAlchemy ORM

In [8]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [9]:
# prep db file to connect
engine = create_engine("sqlite:///hawaii.sqlite")

In [10]:
# Get the name of the table. 
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [11]:
columns = inspector.get_columns('measurement')
for column in columns:
    print(column)

{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'station', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'date', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'prcp', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'tobs', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}


In [12]:
columns = inspector.get_columns('station')
for column in columns:
    print(column)

{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'station', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'name', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'latitude', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'longitude', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'elevation', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}


In [13]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [14]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [17]:
# Get the column names for the table
column_names = inspector.get_columns('measurement')
for column_name in column_names:
    print(column_name["name"])

id
station
date
prcp
tobs


In [15]:
# Get the column names for the table
column_names = inspector.get_columns('station')
for column_name in column_names:
    print(column_name["name"])

id
station
name
latitude
longitude
elevation


Save References to Each Table
In order to reference a specific class, we use Base.classes.<class name>. For example, if we wanted to reference the station class, we would use Base.classes.station.

Since it can be rather cumbersome to type Base.classes every time we want to reference the measurement or station classes, we can give the classes new variable names. In this case, we will create new references for our Measurement class and Station class.

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
results = session.query(Measurement.date, Measurement.prcp).all()

In [ ]:
for result in results: 
    print(result)
# appears to be tuple dtype

This code specifies the most recent date, but we want to calculate the date one year back. To do this, add the dt.timedelta() function to the previous line of code. This function allows us to trace back a certain number of days. In this case, we want to go back 365 days. 

In [ ]:
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

In [ ]:
# create variable to store query results
results = []

In [ ]:
# add session
results = session.query(Measurement.date, Measurement.prcp)
print(results.all())

In [ ]:
# filter out data older than a year from the last record
# use .all() to extract all the query results into a list
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_year).all()
print(results)

In [ ]:
# 9.2.2 save query to DF
df = pd.DataFrame(results, columns=['date', 'precipitation'])

In [ ]:
df.set_index(df['date'], inplace=True)

In [ ]:
df

because we are using the date as the index, the DataFrame has two date columns, which is confusing. So we'll print the DataFrame without the index so we can see just the date and precipitation.

For this task, we'll need to use a slightly different print statement. First we'll convert the DataFrame to strings, and then we'll set our index to "False." This will allow us to print the DataFrame without the index

In [ ]:
print(df.to_string(index=False))

In [ ]:
df.sort_index()
print(df.sort_index().to_string(index=False))

In [ ]:
df.plot()

In [ ]:
df.describe()

In [ ]:
# write a query to get the number of stations in our dataset
# use previous session, func.count() and .all() to return the list
session.query(func.count(Station.station)).all()

In [ ]:
# 9.3.2 determine most active station
# 
session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [ ]:
# 9.3.3 temps, using most active station
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
filter(Measurement.station == 'USC00519281').all()

In [ ]:
# 9.3.4 get total observations from most active station for recent year
results = session.query(Measurement.tobs).\
filter(Measurement.station == 'USC00519281').\
filter(Measurement.date >= prev_year).all()
print(results)

In [ ]:
# convert to df
df = pd.DataFrame(results, columns=['tobs'])
print(df)

In [ ]:
print(df.to_string(index=False))

In [ ]:
# plot histogram of bins

df.plot.hist(bins=12)
plt.tight_layout()

In [ ]:
df.plot.hist(bins=5)
plt.tight_layout()

In [ ]:
df.plot.hist(bins=20)
plt.tight_layout()

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
#Starting from the last data point in the database. 

# Calculate the date one year from the last date in data set.

# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


In [ ]:
# How many stations are available in this dataset?


In [ ]:
# What are the most active stations?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


In [ ]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates


# Challenge